In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 55 kB 1.9 MB/s 
     |████████████████████████████████| 8.2 MB 8.9 MB/s 
     |████████████████████████████████| 371 kB 55.2 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=854575 sha256=b04d2902f72ecb463ec3c36f448f081fb149c715ea0bbc70506e0a11e1089f7b
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [3]:
import random
import re
import string
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import ToTensor
import warnings 
from sklearn.linear_model import LogisticRegression
from PIL import Image
from IPython.display import clear_output
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from typing import List
from sklearn.metrics import classification_report
import pickle
from typing import List
import nltk
import string
import pymorphy2
import codecs

In [4]:
warnings.filterwarnings(action='ignore',category=UserWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='gensim')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='smart_open') 
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='sklearn')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='scipy')    

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
data = pd.read_excel('/content/drive/MyDrive/tenderhack/dataset_base.xlsx')
data.head()

,Идентификатор СТЕ,Наименование СТЕ,Категория,Описание,Характеристики СТЕ,Регионы поставки,Кол-во заключенных контрактов,Поставщики,Страна происхождения,Другая продукция в контрактах,Просмотры,Идентификатор КПГЗ,Код КПГЗ,Модель,Цена
0,20528973,"Светодиодная лента 2835, ULTRA, 12 Вольт, 60 l...",Лампы светодиодные,NaN,"[{""Name"":""Цвет свечения"",""Id"":-506425117,""Valu...","[{""Name"":""Москва""}]",NaN,"[{""SupplierId"":2310783,""Name"":""ООО \""ЛЮМЕНВАТТ...",NaN,NaN,1,12106442.0,01.10.05.02.04,2835ULTRA,NaN
1,34860341,Светодиодный светильник Geniled ЛПО 1200?180?...,Настенно-потолочные светильники,NaN,"[{""Name"":""Вид климатического исполнения"",""Id"":...","[{""Name"":""Свердловская""},{""Name"":""Москва""}]",1.0,"[{""SupplierId"":1559525,""Name"":""Общество с огра...",РОССИЯ,NaN,41,12114082.0,01.10.05.03.01.05,ЛПО 1200?180?40 40Вт 5000К Опал,NaN
2,34172198,Елка пристенно-угловая Графская премиум 210 см...,Предметы внутреннего интерьера,NaN,"[{""Name"":""Цвет"",""Id"":340085190,""Value"":""Зелены...","[{""Name"":""Москва""},{""Name"":""Все регионы""}]",NaN,"[{""SupplierId"":1348988,""Name"":""ИП Чепало Оксан...",NaN,NaN,5,68093.0,01.20.08,Елка пристенно-угловая Графская премиум 210 см.,NaN
3,1318014,Картридж HP CE278A/Canon 728 LJ P1560/1566/160...,Расходные материалы и комплектующие для лазерн...,NaN,"[{""Name"":""Тип расходника"",""Id"":317455168,""Valu...","[{""Name"":""Москва""},{""Name"":""Московская""}]",9.0,"[{""SupplierId"":1111141,""Name"":""Общество с огра...",NaN,"[{""OtherSkuId"":1398583,""OtherSkuName"":""013R006...",6,12105204.0,01.13.13.01.01,SF-CE278(2K),NaN
4,1251411,Штамп самонаборный Pr.C20/3-Set 38х14 (аналог ...,Штемпели,NaN,"[{""Name"":""Наличие рамки"",""Id"":366100137,""Value...","[{""Name"":""Белгородская""},{""Name"":""Марий Эл""},{...",71.0,"[{""SupplierId"":1100019,""Name"":""ООО \""ЦТО\"""",""I...",АВСТРИЯ,"[{""OtherSkuId"":1171023,""OtherSkuName"":""Berling...",72,12120460.0,01.15.06.07,Штамп самонаборный пласт. 3стр. Pr.C20/3-Set 3...,"[{""Cost"":454.67000},{""Cost"":620.00000},{""Cost""..."


In [7]:
len(data)

200000

In [8]:
data['Категория'].value_counts()

Расходные материалы и комплектующие для лазерных принтеров и МФУ                                  6519
Ручки канцелярские                                                                                3230
Одежда специальная для защиты от общих производственных загрязнений и механических воздействий    3015
Папки пластиковые                                                                                 1994
Учебники печатные общеобразовательного назначения                                                 1951
                                                                                                  ... 
Установки вентиляционные                                                                             1
Расходные материалы и комплектующие для дупликаторов                                                 1
Сигналы                                                                                              1
Мочеприемник закрытый неносимый, стерильный                              

In [ ]:
data.dtypes.value_counts()

object     11
int64       2
float64     2
dtype: int64

In [ ]:
type(data['Характеристики СТЕ'][0])

str

In [13]:
import json
characts = []
for i in data['Характеристики СТЕ']:
  try:
    characts.append(json.loads(i))
  except:
    pass


In [14]:
print(characts[0])

[{'Name': 'Цвет свечения', 'Id': -506425117, 'Value': 'Холодный'}, {'Name': 'Мощность', 'Id': -506424125, 'Value': '7.00000', 'Unit': 'Вт'}, {'Name': 'Напряжение', 'Id': -506424124, 'Value': '12', 'Unit': 'В'}, {'Name': 'Яркость', 'Id': -506425116, 'Value': '760', 'Unit': 'лм'}, {'Name': 'Плотность led на метр', 'Id': -506425119, 'Value': '300', 'Unit': 'шт'}, {'Name': 'Размер светодиодов', 'Id': -506425118, 'Value': '283', 'Unit': 'smd'}]


In [15]:
all_characts = {}
charact_counter = {}
for charact_set in characts:
  for charact in charact_set:
    name = charact['Name'].strip().lower()
    if name[0] == '"':
      name = name[1:-1]
    elif name[0] == "'":
      name = name[1:]
    if name not in all_characts:
      all_characts[name] = set()
      charact_counter[name] = 0
    charact_counter[name]+=1
    try:
      all_characts[name].add(charact['Value'].strip().lower())
    except KeyError:
      pass


In [16]:
print(len(all_characts))
print(list(all_characts.keys())[:20])

57717
['цвет свечения', 'мощность', 'напряжение', 'яркость', 'плотность led на метр', 'размер светодиодов', 'вид климатического исполнения', 'вид рассеивателя', 'коэффициент мощности', 'вид товаров', 'вид энергетического оборудования', 'габаритные размеры (шхгхв)', 'световой поток', 'цветовая температура', 'гарантия', 'степень защиты', 'цвет', 'металлическая подставка', 'материал', 'вид ели']


In [17]:
print(list(sorted(all_characts.keys()))[:20])

['+частота процессора', '- заряд', '- разряд', '. рн 10% водного раствора мыла', '1  – пропанол', '1 disilet с отслаиваемой оболочкой', '1 xro силиконовый катетер  4,2 fr', '1 замес', '1 порт', '1 радиотрубка', '1 туннельная игла', '1,62', '1-й класс', '1-й тип коннектора', '10/100base-tx с поддержкой poe', '100 внутренних абонентов. расширяется до 200 с помощью dsp модуля d3', '100% арабика', '10base-t / 100base-tx / 1000base-t', '12 модулей размером', '12. способ измерения температуры']


In [18]:
good_keys = [key for key in charact_counter if charact_counter[key]>=10]
len(good_keys)

10555

In [19]:
sorted(good_keys)[:20]

['100% арабика',
 '3 в 1',
 '3d эффект',
 '3d-датчик ускорения',
 '3d-печать',
 '3g',
 '4g',
 '4g lte',
 '[in] номинальный ток',
 'access control list',
 'adf (податчик оригиналов)',
 'aoh',
 'atx',
 'av выход',
 'bec',
 'bio-средство',
 'bios',
 'bluetooth',
 'bluetooth встроенный',
 'classless inter-domain routing (cidr)']

In [20]:
for i in sorted(good_keys)[:20]:
  print(i, all_characts[i])

100% арабика {'нет', 'да'}
3 в 1 {'нет', 'да'}
3d эффект {'объемная', 'эффект движения'}
3d-датчик ускорения {'нет', 'да'}
3d-печать {'объмная с движением', 'объемная'}
3g {'нет', 'да'}
4g {'нет', 'да'}
4g lte {'нет', 'да'}
[in] номинальный ток {'95', '125', '10', '7', '2', '40', '16', '32', '9', '100', '25', '250', '80', '315', '1', '160', '6', '63'}
access control list {'нет', 'поддерживается, до 50 списков доступа'}
adf (податчик оригиналов) {'есть', 'односторонний', 'обычный', 'да', 'двусторонний', 'dadf  на 50 листов', 'наличие', '1', 'нет'}
aoh {'есть', 'да'}
atx {'нет', 'да'}
av выход {'нет', 'да'}
bec {'3.60000', '52.00000', '40.00000', '8.00000', '18.00000', '16.00000', '20.00000', '15', '15.00000', '3.00000', '5.00000', '30.00000', '25.00000', '4.00000', '10.00000', '1.00000'}
bio-средство {'нет', 'да'}
bios {'ami', 'efi ami bios, 1x 128 мбит', 'uefi ami bios, 2x128 мбит', '128 mb flash rom, uefi ami bios', 'efi ami bios, 2x 128 мбит', 'efi ami bios, 128 мбит', 'uefi ami bios

In [21]:
data_dict = {}
for iterr, st in data.iterrows():
  data_dict[st['Идентификатор СТЕ']] = st

In [42]:
connected = {}
for iterr, st in data.iterrows():
  if st['Категория'] not in connected:
    connected[st['Категория']] = {}
  if not pd.isna(st['Другая продукция в контрактах']) and len(st['Другая продукция в контрактах'].strip())>0:
    st_others = st['Другая продукция в контрактах']
    edge_inda = -st_others[::-1].index("}")
    if edge_inda!=0:
      st_others = st_others[:edge_inda]+']'
    print(st_others)
    st_data = json.loads(st_others)
    for prod in st_data:
      try:
        connected_cat = data_dict[prod['OtherSkuId']]['Категория']
        if connected_cat not in connected[st['Категория']]:
          connected[st['Категория']][connected_cat] = 0
        connected[st['Категория']][connected_cat] += 1
      except KeyError:
        pass

      

[{"OtherSkuId":1398583,"OtherSkuName":"013R00670 Копи-картридж WC 5019\/5021 (80 000 стр.)","Quantity":2.00000},{"OtherSkuId":34609722,"OtherSkuName":"Картридж 057 для принтера Canon совместимый","Quantity":30.00000},{"OtherSkuId":18371982,"OtherSkuName":"Картридж Cactus  718 (CS-C718BK) для Canon i-SENSYS LBP7200Cdn\/HP CLJ CP2025.","Quantity":8.00000},{"OtherSkuId":18371657,"OtherSkuName":"Картридж Cactus Cartridge 718 (CS-C718C) для Canon i-SENSYS LBP7200Cdn\/HP CLJ CP2025 (2800 стр.)","Quantity":8.00000},{"OtherSkuId":18372867,"OtherSkuName":"Картридж Cactus Cartridge 718 (CS-C718M) для Canon i-SENSYS LBP7200Cdn\/HP CLJ CP2025.","Quantity":8.00000},{"OtherSkuId":18372754,"OtherSkuName":"Картридж Cactus Cartridge 718 (CS-C718Y) для Canon i-SENSYS LBP7200Cdn\/HP CLJ CP2025 (2800 стр.)","Quantity":8.00000},{"OtherSkuId":1280137,"OtherSkuName":"Картридж Canon EP-27\/EP-26(совм. Colouring) LBP-3200\/ MF3110\/3228\/3240\/5630\/57xx (2500стр.)","Quantity":15.00000},{"OtherSkuId":17825394,

JSONDecodeError: ignored